In [65]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from pgmpy.estimators import BDeuScore, K2Score, BicScore, ExhaustiveSearch, HillClimbSearch, BayesianEstimator, MaximumLikelihoodEstimator
from pgmpy.models import BayesianModel
from pgmpy.inference import BeliefPropagation

## Data import and variable selection
- Import geosmin, microcystin and cyanobacteria abundance datasets
- Select columns according to importance ranking<br>
Geosmin:<br>
![image](var_geo.gif)<br>
Cyanobacteria:<br>
![image](var_cyano.gif)<br>
Microcystin:<br>
![image](var_mc.gif)<br>

In [33]:
df_g = pd.read_csv("geosmin.csv")[['Date','SSC','NO','cos','DO','Si','TKN','Geo']]
df_m = pd.read_csv("microcystin.csv")[['Date','Temp','sin','DO','Chl-a','TP','Fe','MC']]
df_c = pd.read_csv("cyanobacteria_abundance.csv")[['Date','Chl-a','sin','TKN','DP','Si','Temp','Cyano']]
df_sp = pd.read_csv("solar&prec.csv").drop(["LAT","LON","YEAR","MO","DY"], axis=1)

## Data merge

In [34]:
df_geo = df_sp.merge(df_g,on = 'Date',how = 'inner').drop(['Date'],axis=1)
df_mc = df_sp.merge(df_m,on = 'Date',how = 'inner').drop(['Date'],axis=1)
df_cyano = df_sp.merge(df_c,on = 'Date',how = 'inner').drop(['Date'],axis=1)
df_geo

,PRECTOT,ALLSKY_SFC_SW_DWN,SSC,NO,cos,DO,Si,TKN,Geo
0,0.00,22.07,12.8,0.030,-0.81,5.91,1.44,0.590,3.7
1,0.02,22.86,10.9,0.070,-0.44,6.70,7.04,0.556,4.8
2,0.18,13.21,3.8,0.010,0.76,14.42,8.55,0.556,22.0
3,0.96,15.84,2.6,0.170,-0.98,6.16,5.95,0.619,63.0
4,0.36,22.03,18.1,0.030,-1.00,7.98,7.56,0.786,7.0
...,...,...,...,...,...,...,...,...,...
179,0.00,10.37,3.0,0.005,0.98,14.64,162.00,0.469,7.6
180,0.00,14.24,4.0,0.005,0.76,13.65,0.71,0.697,5.0
181,0.02,19.74,3.0,0.005,0.37,13.57,0.71,0.633,2.7
182,0.04,12.85,40.0,0.034,-0.23,8.30,10.14,0.738,0.5


## Predict the missing data
Predicts states of all the missing variables.

## Split the data into training/test set
75% of the response variable data were used in training. The data was selected at random.

In [78]:
y_geo = df_g.loc[:, 'Geo']
x_geo = df_geo.loc[:, 'PRECTOT':'TKN']
y_mc = df_m.loc[:, 'MC']
x_mc = df_mc.loc[:, 'PRECTOT':'Fe']
y_cyano = df_c.loc[:, 'Cyano']
x_cyano = df_cyano.loc[:, 'PRECTOT':'Temp']
x_geo_train, x_geo_test, y_geo_train, y_geo_test = train_test_split(x_geo, y_geo, train_size=0.75)
x_mc_train, x_mc_test, y_mc_train, y_mc_test = train_test_split(x_mc, y_mc, train_size=0.75)
x_cyano_train, x_cyano_test, y_cyano_train, y_cyano_test = train_test_split(x_cyano, y_cyano, train_size=0.75)
geo_train = pd.concat([x_geo_train, y_geo_train], axis = 1)
mc_train = pd.concat([x_mc_train, y_mc_train], axis = 1)
cyano_train = pd.concat([x_cyano_test, y_cyano_test], axis = 1)
geo_test = pd.concat([x_geo_test, y_geo_test], axis = 1)
mc_test = pd.concat([x_mc_test, y_mc_test], axis = 1)
cyano_test = pd.concat([x_cyano_test, y_cyano_test], axis = 1)
mc_test

,PRECTOT,ALLSKY_SFC_SW_DWN,Temp,sin,DO,Chl-a,TP,Fe,MC
29,0.02,20.02,24.2,-0.91,8.71,41.2,0.112,490.0,2.65
46,0.00,20.92,23.8,-0.95,8.10,18.6,0.110,510.0,0.63
17,0.00,14.54,14.5,-0.90,8.94,8.1,0.070,226.0,0.20
135,0.01,29.20,23.7,0.33,7.44,5.8,0.140,1320.0,0.40
98,7.87,7.21,5.7,0.93,13.83,32.1,0.070,410.0,0.05
65,0.00,19.14,21.6,-1.00,8.26,10.5,0.060,770.0,0.05
155,0.00,16.00,14.5,-0.94,9.21,13.0,0.170,1060.0,0.30
67,0.00,12.44,12.7,-0.82,9.45,3.7,0.090,990.0,0.11
21,0.03,11.02,17.1,0.92,8.48,11.1,0.090,200.0,0.05
31,0.69,14.47,17.1,-0.98,7.92,21.9,0.134,290.0,0.72


## Structure learning
- Score based structure learning using BDeu, K2 and BIC. It computes a score to measure how well the given Bayesian model fits to the data set
- Searching methods include Exhaustive Search(Searches all possible graph with a given set of nodes) and Hill Climb Search(Starts at model start_dag and proceeds by step-by-step network modifications until a local maximum is reached) according to the scorng method supplied.

Given n nodes, 2^(n*(n-1)) graphs need to be searched in Exhaustive Search. It is likely not feasible for n>6. 

In [36]:
#Ramdon sample 20% rows
df_mc_sl = df_mc.sample(frac=0.2)
df_geo_sl = df_geo.sample(frac=0.2)
df_cyano_sl = df_cyano.sample(frac=0.2)

In [37]:
#Use BIC scoring method and HillClimbSearch for microcystin, cyanobacteria abundance and geosmin model
hc_mc = HillClimbSearch(df_mc_sl, scoring_method=BicScore(df_mc))
best_model_mc = hc_mc.estimate()
print(best_model_mc.edges())
hc_geo = HillClimbSearch(df_geo_sl, scoring_method=BicScore(df_geo))
best_model_geo = hc_geo.estimate()
print(best_model_geo.edges())
hc_cyano = HillClimbSearch(df_cyano_sl, scoring_method=BicScore(df_cyano))
best_model_cyano = hc_cyano.estimate()
print(best_model_cyano.edges())

  0%|          | 0/1000000 [00:00<?, ?it/s]

[('PRECTOT', 'TP'), ('PRECTOT', 'Chl-a'), ('ALLSKY_SFC_SW_DWN', 'DO'), ('ALLSKY_SFC_SW_DWN', 'Fe'), ('Temp', 'ALLSKY_SFC_SW_DWN'), ('sin', 'Temp'), ('sin', 'ALLSKY_SFC_SW_DWN'), ('Chl-a', 'sin'), ('TP', 'Chl-a'), ('TP', 'sin'), ('TP', 'Temp'), ('MC', 'Chl-a')]


  0%|          | 0/1000000 [00:00<?, ?it/s]

[('PRECTOT', 'SSC'), ('PRECTOT', 'NO'), ('ALLSKY_SFC_SW_DWN', 'DO'), ('SSC', 'NO'), ('SSC', 'cos'), ('SSC', 'TKN'), ('NO', 'cos'), ('NO', 'Si'), ('cos', 'TKN'), ('Si', 'ALLSKY_SFC_SW_DWN'), ('TKN', 'Si'), ('TKN', 'ALLSKY_SFC_SW_DWN'), ('Geo', 'SSC'), ('Geo', 'cos')]


  0%|          | 14/1000000 [00:08<159:47:06,  1.74it/s]

[('PRECTOT', 'DP'), ('PRECTOT', 'sin'), ('PRECTOT', 'Temp'), ('ALLSKY_SFC_SW_DWN', 'Chl-a'), ('sin', 'TKN'), ('TKN', 'ALLSKY_SFC_SW_DWN'), ('TKN', 'Cyano'), ('DP', 'sin'), ('DP', 'Si'), ('DP', 'Temp'), ('Si', 'TKN'), ('Temp', 'Si'), ('Temp', 'sin')]


## Define the Bayesian model

In [38]:
mc_model = BayesianModel([('PRECTOT', 'TP'), ('PRECTOT', 'Fe'), ('ALLSKY_SFC_SW_DWN', 'DO'), ('Temp', 'Chl-a'), ('Temp', 'ALLSKY_SFC_SW_DWN'), ('sin', 'Temp'), ('Chl-a', 'ALLSKY_SFC_SW_DWN'), ('TP', 'Temp'), ('Fe', 'sin'), ('Fe', 'Chl-a'), ('Fe', 'TP'), ('Fe', 'DO'), ('MC', 'TP'), ('MC', 'Fe')])
geo_model = BayesianModel([('PRECTOT', 'NO'), ('PRECTOT', 'cos'), ('ALLSKY_SFC_SW_DWN', 'DO'), ('ALLSKY_SFC_SW_DWN', 'Geo'), ('SSC', 'cos'), ('NO', 'cos'), ('NO', 'SSC'), ('NO', 'TKN'), ('cos', 'TKN'), ('cos', 'ALLSKY_SFC_SW_DWN'), ('DO', 'Si'), ('TKN', 'ALLSKY_SFC_SW_DWN')]
)
cyano_model = BayesianModel(
[('PRECTOT', 'DP'), ('PRECTOT', 'Cyano'), ('ALLSKY_SFC_SW_DWN', 'Chl-a'), ('ALLSKY_SFC_SW_DWN', 'Si'), ('Chl-a', 'Temp'), ('Chl-a', 'sin'), ('TKN', 'ALLSKY_SFC_SW_DWN'), ('DP', 'Cyano'), ('DP', 'TKN'), ('Cyano', 'TKN'), ('Cyano', 'ALLSKY_SFC_SW_DWN')]
)
print(mc_model.nodes())
print(geo_model.nodes())
print(cyano_model.nodes())

['PRECTOT', 'TP', 'Fe', 'ALLSKY_SFC_SW_DWN', 'DO', 'Temp', 'Chl-a', 'sin', 'MC']
['PRECTOT', 'NO', 'cos', 'ALLSKY_SFC_SW_DWN', 'DO', 'Geo', 'SSC', 'TKN', 'Si']
['PRECTOT', 'DP', 'Cyano', 'ALLSKY_SFC_SW_DWN', 'Chl-a', 'Si', 'Temp', 'sin', 'TKN']


## Directed graph visualization
- Microcystin:<br>
![image](mc_directed_graph.png)
- Geosmin:<br>
![image](geo_directed_graph.png)
- Cyanobacteria:<br>
![image](cyano_directed_graph.png)

## Parameter learning
- Maximum Likelihood Estimator(usually has the problem of overfitting to the data)
- Bayesian Estimator(starts with already existing prior conditional probability distributions(CPDs), that express our beliefs about the variables before the data was observed)

In [62]:
#Nodes: ['PRECTOT', 'TP', 'Fe', 'ALLSKY_SFC_SW_DWN', 'DO', 'Temp', 'Chl-a', 'sin', 'MC']
mc_model_est = MaximumLikelihoodEstimator(mc_model, df_mc_sl)
cpd_mc_TP = mc_model_est.estimate_cpd('TP')
cpd_mc_sin = mc_model_est.estimate_cpd('sin')
cpd_mc_Fe = mc_model_est.estimate_cpd('Fe')
cpd_mc_Temp = mc_model_est.estimate_cpd('Temp')
cpd_mc_Solar = mc_model_est.estimate_cpd('ALLSKY_SFC_SW_DWN')
cpd_mc_DO = mc_model_est.estimate_cpd('DO')
cpd_mc_Precipitation= mc_model_est.estimate_cpd('PRECTOT')
cpd_mc_chla = mc_model_est.estimate_cpd('Chl-a')
cpd_mc = mc_model_est.estimate_cpd('MC')
mc_model.add_cpds(cpd_mc_TP, cpd_mc_sin, cpd_mc_Fe, cpd_mc_Temp, cpd_mc_Solar, cpd_mc_DO, cpd_mc_Precipitation, cpd_mc_chla, cpd_mc)

In [63]:
#Nodes: ['PRECTOT', 'NO', 'cos', 'ALLSKY_SFC_SW_DWN', 'DO', 'Geo', 'SSC', 'TKN', 'Si']
geo_model_est = MaximumLikelihoodEstimator(geo_model, df_geo_sl)
cpd_geo_TKN = geo_model_est.estimate_cpd('TKN')
cpd_geo_cos = geo_model_est.estimate_cpd('cos')
cpd_geo_Si = geo_model_est.estimate_cpd('Si')
cpd_geo_NO = geo_model_est.estimate_cpd('NO')
cpd_geo_Solar = geo_model_est.estimate_cpd('ALLSKY_SFC_SW_DWN')
cpd_geo_DO = geo_model_est.estimate_cpd('DO')
cpd_geo_Precipitation= geo_model_est.estimate_cpd('PRECTOT')
cpd_geo_SSC = geo_model_est.estimate_cpd('SSC')
cpd_geo = geo_model_est.estimate_cpd('Geo')
geo_model.add_cpds(cpd_geo_TKN, cpd_geo_cos, cpd_geo_Si, cpd_geo_NO, cpd_geo_Solar, cpd_geo_DO, cpd_geo_Precipitation, cpd_geo_SSC, cpd_geo)

In [70]:
#Nodes: ['PRECTOT', 'DP', 'Cyano', 'ALLSKY_SFC_SW_DWN', 'Chl-a', 'Si', 'Temp', 'sin', 'TKN']
cyano_model_est = MaximumLikelihoodEstimator(cyano_model, df_cyano_sl)
cpd_cyano_TKN = cyano_model_est.estimate_cpd('TKN')
cpd_cyano_sin = cyano_model_est.estimate_cpd('sin')
cpd_cyano_Si = cyano_model_est.estimate_cpd('Si')
cpd_cyano_Temp = cyano_model_est.estimate_cpd('Temp')
cpd_cyano_Solar = cyano_model_est.estimate_cpd('ALLSKY_SFC_SW_DWN')
cpd_cyano_DP = cyano_model_est.estimate_cpd('DP')
cpd_cyano_Precipitation= cyano_model_est.estimate_cpd('PRECTOT')
cpd_cyano_chla = cyano_model_est.estimate_cpd('Chl-a')
cpd_Cyano = cyano_model_est.estimate_cpd('Cyano')
cyano_model.add_cpds(cpd_cyano_TKN, cpd_cyano_sin, cpd_cyano_Si, cpd_cyano_Temp, cpd_cyano_Solar, cpd_cyano_DP, cpd_cyano_Precipitation, cpd_cyano_chla, cpd_Cyano)

## Marginal probability distribution calculation
- Variable Elimination
- Belief Propagation

In [71]:
mc_bp = BeliefPropagation(mc_model)
geo_bp = BeliefPropagation(geo_model)
cyano_bp = BeliefPropagation(cyano_model)

## Model Validation
- random selection in testing dataset
- cross validation

In [85]:
#random select a row in testing dataset
test1 = mc_test.sample(n=1)
mc_bp.map_query(variables=['MC'], evidence={'PRECTOT': 0.0, 'ALLSKY_SFC_SW_DWN': 26.09, 'Temp': 29.4,
                                          'DO':7.16, 'Chl-a':16.4, 'TP':0.23, 'Fe':870.0})

/opt/anaconda3/lib/python3.8/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:518: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  warn(


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [83]:
test1

,PRECTOT,ALLSKY_SFC_SW_DWN,Temp,sin,DO,Chl-a,TP,Fe,MC
121,0.0,26.09,29.4,-0.5,7.16,16.4,0.23,870.0,2.16
